# 🚀 Day 2-1: 분류 모델의 성적표, 핵심 평가지표 완전 정복 💯

회귀 모델이 연속적인 값을 예측했다면, **분류(Classification)** 모델은 데이터가 어떤 범주(Category)에 속하는지 예측합니다. 예를 들어 "이 이메일은 스팸인가, 아닌가?", "사진 속 동물은 고양이인가, 강아지인가, 토끼인가?", "고객이 서비스를 이탈할 것인가, 유지할 것인가?" 와 같은 질문에 답하는 것이죠.

모델을 만들고 나면 우리는 회귀 파트에서처럼 "그래서 이 모델, 얼마나 잘하는데?"라는 질문을 던져야 합니다. 하지만 분류 문제의 평가는 회귀와는 다른 척도를 사용해야 합니다. 단순히 '정확히 맞춘 개수'만으로는 모델의 성능을 제대로 파악하기 어려운 경우가 많기 때문입니다.

특히 **클래스 불균형(Class Imbalance)** 상황, 즉 한쪽 범주의 데이터가 다른 쪽보다 월등히 많은 경우(예: 전체 카드 거래 중 99.9%가 정상, 0.1%가 사기 거래)에 정확도만 믿고 모델을 평가하면 큰 함정에 빠질 수 있습니다.

이번 시간에는 분류 모델의 성능을 다각도로 측정하고 올바르게 해석하기 위한 핵심 평가지표들, **혼동 행렬(Confusion Matrix)부터 Accuracy, Precision, Recall, F1-score, 그리고 ROC-AUC**까지 깊이 있게 알아보겠습니다. 각 지표가 어떤 이야기를 들려주는지, 문제 상황에 따라 어떤 지표를 선택해야 하는지 명확히 이해하는 것을 목표로 합니다.

-----

### 1. 혼동 행렬(Confusion Matrix): 모든 평가의 시작

**혼동 행렬**은 이름 그대로 모델이 얼마나 '혼동'하고 있는지를 보여주는 행렬표입니다. 실제 정답(True Label)과 모델의 예측(Predicted Label)을 비교하여, 모델의 예측이 4가지 경우 중 어디에 속하는지 보여줍니다. 모든 분류 평가지표는 이 혼동 행렬로부터 계산됩니다.

#### 🧠 개념 이해하기

이진 분류(Binary Classification) 상황(예: Yes/No, 1/0, Positive/Negative)을 기준으로 살펴보겠습니다. 여기서 'Positive'는 우리가 찾으려는 주된 대상(예: 질병, 사기, 이탈)을 의미합니다.

  * **TP (True Positive, 진양성)**: 실제 'Positive'를 'Positive'로 **올바르게** 예측. (정답\!)
  * **TN (True Negative, 진음성)**: 실제 'Negative'를 'Negative'로 **올바르게** 예측. (정답\!)
  * **FP (False Positive, 위양성)**: 실제 'Negative'를 'Positive'로 **틀리게** 예측. (1종 오류)
  * **FN (False Negative, 위음성)**: 실제 'Positive'를 'Negative'로 **틀리게** 예측. (2종 오류)

예를 들어, 암 진단 모델의 경우:

  * **TP**: 암 환자를 암이라고 진단함 (가장 좋은 시나리오)
  * **TN**: 건강한 사람을 건강하다고 진단함 (좋은 시나리오)
  * **FP**: 건강한 사람을 암 환자라고 진단함 (환자는 놀라겠지만, 추가 검사로 바로잡을 수 있음)
  * **FN**: 암 환자를 건강하다고 진단함 (치료 시기를 놓칠 수 있는 **가장 치명적인** 시나리오)

이처럼 각 오류(FP, FN)가 가지는 비용과 중요도는 문제 상황에 따라 크게 달라집니다.

#### 💻 코드 예시

`scikit-learn`의 `confusion_matrix`를 사용하면 혼동 행렬을 쉽게 만들 수 있습니다. 시각화에는 `plotly.express`의 `imshow`를 활용해봅시다.

In [1]:
import numpy as np
from sklearn.metrics import confusion_matrix
import plotly.express as px

# 실제 정답: 5명은 '이탈 안함'(0), 5명은 '이탈함'(1)
y_true = np.array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])
# 모델의 예측
y_pred = np.array([0, 0, 1, 0, 0, 1, 0, 1, 1, 1])

# 혼동 행렬 계산
# 결과: [[TN, FP], [FN, TP]]
cm = confusion_matrix(y_true, y_pred)

print("혼동 행렬:\n", cm)


혼동 행렬:
 [[4 1]
 [1 4]]


In [2]:

# 혼동 행렬 시각화
fig = px.imshow(cm, text_auto=True,
                labels=dict(x="Predicted Label", y="True Label", color="Count"),
                x=['Negative(0)', 'Positive(1)'],
                y=['Negative(0)', 'Positive(1)'],
                title="Confusion Matrix")
fig.show()

위 예시에서 TN=4, FP=1, FN=1, TP=4 임을 시각적으로 확인할 수 있습니다.

-----

### 2\. Accuracy (정확도): 가장 직관적이지만 위험한 지표

#### 🧠 개념 이해하기

\*\*정확도(Accuracy)\*\*는 가장 직관적인 평가지표로, 전체 샘플 중 모델이 올바르게 예측한 샘플의 비율을 나타냅니다.

$$\text{Accuracy} = \frac{\text{TP} + \text{TN}}{\text{TP} + \text{TN} + \text{FP} + \text{FN}}$$

"이 모델은 85%의 정확도를 보여"와 같이 이해하기 쉽지만, 치명적인 맹점을 가지고 있습니다. 바로 \*\*클래스 불균형(imbalanced data)\*\*에 매우 취약하다는 점입니다.

**정확도의 함정 (Accuracy Paradox)**
고객 1000명 중 990명은 이탈하지 않고(Negative), 10명만 이탈(Positive)하는 데이터가 있다고 가정해봅시다. 만약 어떤 모델이 모든 고객에 대해 "이탈하지 않는다"라고만 예측한다면 어떻게 될까요?

  * TP = 0, FP = 0, TN = 990, FN = 10
  * Accuracy = (0 + 990) / (0 + 0 + 990 + 10) = 990 / 1000 = **99%**

정확도는 99%로 매우 높지만, 정작 우리가 찾고 싶었던 '이탈 고객'은 단 한 명도 찾아내지 못하는, 사실상 아무 쓸모없는 모델입니다. 이처럼 불균형 데이터에서는 정확도가 모델의 성능을 심각하게 왜곡할 수 있습니다.

#### 💻 코드 예시

위의 불균형 시나리오를 코드로 확인해 보겠습니다.

In [3]:
from sklearn.metrics import accuracy_score

# 1000명 중 10명만 Positive(1)인 불균형 데이터
y_true_imbalanced = np.array([0] * 990 + [1] * 10)
# 모든 것을 Negative(0)로 예측하는 무능한 모델
y_pred_dummy = np.zeros(1000)

accuracy = accuracy_score(y_true_imbalanced, y_pred_dummy)
print(f"불균형 데이터에 대한 무능한 모델의 정확도: {accuracy:.2f}")

cm_imbalanced = confusion_matrix(y_true_imbalanced, y_pred_dummy)
print("혼동 행렬:\n", cm_imbalanced)

불균형 데이터에 대한 무능한 모델의 정확도: 0.99
혼동 행렬:
 [[990   0]
 [ 10   0]]


#### ✏️ 연습문제 1

어떤 모델의 혼동 행렬이 아래와 같을 때, 이 모델의 정확도를 직접 계산해보세요.

  * TP = 60
  * FP = 10
  * TN = 120
  * FN = 10

-----

### 3. Precision (정밀도) & Recall (재현율): 관점의 차이

정확도의 한계를 극복하기 위해 등장한 지표가 바로 정밀도와 재현율입니다. 두 지표는 Positive 클래스에 초점을 맞추지만, 서로 다른 관점에서 모델을 평가합니다.

#### 🧠 개념 이해하기

**1. Precision (정밀도)**

> "모델이 'Positive'라고 예측한 것들 중에서, 얼마나 진짜 'Positive'였는가?"

$$\text{Precision} = \frac{\text{TP}}{\text{TP} + \text{FP}}$$

정밀도는 **FP (False Positive)** 를 줄이는 것이 중요할 때 높은 값을 가집니다. 즉, 모델의 예측을 믿고 Positive라고 판단했을 때의 확신도입니다.

  * **사용 예시**: **스팸 메일 필터**. 일반 메일(Negative)을 스팸(Positive)으로 잘못 분류(FP)하면 중요한 메일을 놓치게 됩니다. 따라서 스팸으로 예측하는 것은 매우 신중해야 하므로, 정밀도가 중요합니다. `Precision`이 0.95라면, 스팸으로 분류된 메일 100개 중 95개는 진짜 스팸이라는 의미입니다.

**2. Recall (재현율, 민감도)**

> "실제 'Positive'인 것들 중에서, 모델이 얼마나 'Positive'라고 맞췄는가?"

$$\text{Recall} = \frac{\text{TP}}{\text{TP} + \text{FN}}$$

재현율은 **FN (False Negative)** 를 줄이는 것이 중요할 때 높은 값을 가집니다. 즉, 실제 Positive 샘플을 놓치지 않고 얼마나 잘 찾아내는지를 나타냅니다.

  * **사용 예시**: **암 진단 모델**. 실제 암 환자(Positive)를 건강하다(Negative)고 잘못 진단(FN)하면 생명이 위험할 수 있습니다. 따라서 한 명의 환자라도 놓치지 않는 것이 중요하므로, 재현율이 매우 중요합니다. `Recall`이 0.95라면, 실제 암 환자 100명 중 95명을 모델이 성공적으로 찾아냈다는 의미입니다.

**정밀도-재현율 트레이드오프 (Precision-Recall Trade-off)**
정밀도와 재현율은 일반적으로 반비례 관계를 가집니다. 한쪽을 높이려고 하면 다른 한쪽이 낮아지는 경향이 있습니다. 이는 모델이 Positive/Negative를 판단하는 **임계값(Threshold)** 을 조정함으로써 발생합니다.

  * **임계값을 높이면 (더 확실할 때만 Positive로 예측)**: FP가 줄어들어 **정밀도↑**, 하지만 TP도 줄어들 수 있어 **재현율↓**
  * **임계값을 낮추면 (조금만 가능성 있어도 Positive로 예측)**: FN이 줄어들어 **재현율↑**, 하지만 FP가 늘어나 **정밀도↓**

따라서 문제의 상황에 맞춰 두 지표 사이의 적절한 균형점을 찾는 것이 중요합니다.

#### 💻 코드 예시

`scikit-learn`을 사용하여 정밀도와 재현율을 계산해봅시다.

In [4]:
from sklearn.metrics import precision_score, recall_score

# 암 진단 예시 (FN이 치명적)
y_true_cancer = np.array([0, 1, 1, 0, 1, 0, 0, 1, 0, 1]) # 실제 환자 5명
y_pred_cancer = np.array([0, 1, 0, 0, 1, 1, 0, 1, 0, 1]) # 1명을 놓침(FN=1), 1명을 오진(FP=1)

precision = precision_score(y_true_cancer, y_pred_cancer)
recall = recall_score(y_true_cancer, y_pred_cancer)

print(f"암 진단 모델 정밀도: {precision:.2f}") # 5번 Positive 예측 중 4번이 진짜 -> 4/5 = 0.8
print(f"암 진단 모델 재현율: {recall:.2f}") # 실제 환자 5명 중 4명을 찾아냄 -> 4/5 = 0.8

암 진단 모델 정밀도: 0.80
암 진단 모델 재현율: 0.80


#### ✏️ 연습문제 2 & 3

연습문제 1의 혼동 행렬(TP=60, FP=10, TN=120, FN=10)을 사용하여 **정밀도(Precision)** 와 **재현율(Recall)** 을 각각 계산해보세요.

-----

### 4\. F1 Score: 정밀도와 재현율의 조화로운 평균

정밀도와 재현율은 모두 중요하지만, 두 숫자를 계속해서 비교하는 것은 번거롭습니다. **F1 Score**는 정밀도와 재현율의 **조화 평균(Harmonic Mean)** 을 사용하여 두 지표를 하나의 숫자로 요약해줍니다.

#### 🧠 개념 이해하기

$$\text{F1 Score} = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}$$

왜 산술 평균이 아닌 조화 평균을 사용할까요? 조화 평균은 두 값 중 어느 한쪽이 극단적으로 낮으면 전체 점수도 낮아지는 특징이 있습니다. 따라서 F1 Score는 정밀도와 재현율이 모두 균형 있게 높을 때만 높은 값을 가집니다. 클래스 불균형이 심한 데이터에서 모델의 성능을 정확하게 평가할 때 매우 유용합니다.

  * `Precision=1.0, Recall=0.1` -\> 산술평균=0.55, **F1 Score=0.18**
  * `Precision=0.6, Recall=0.5` -\> 산술평균=0.55, **F1 Score=0.55**

#### 💻 코드 예시

In [5]:
from sklearn.metrics import f1_score

# 위 암 진단 모델 예시 재사용
precision = 0.8
recall = 0.8
f1 = f1_score(y_true_cancer, y_pred_cancer)

print(f"암 진단 모델 F1 Score: {f1:.2f}")

암 진단 모델 F1 Score: 0.80


In [7]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.80      0.80         5
           1       0.80      0.80      0.80         5

    accuracy                           0.80        10
   macro avg       0.80      0.80      0.80        10
weighted avg       0.80      0.80      0.80        10



#### ✏️ 연습문제 4

연습문제 2와 3에서 구한 정밀도와 재현율 값을 사용하여 **F1 Score**를 계산해보세요.

-----

### 5\. 다중 클래스(Multiclass) 분류로의 확장

지금까지는 이진 분류를 다루었지만, 분류 문제는 "고양이 vs 강아지 vs 토끼"처럼 3개 이상의 클래스를 가질 수도 있습니다. 이 경우, 평가지표는 어떻게 계산할까요? 기본적으로 각 클래스에 대해 '이 클래스인가? 아닌가?'의 이진 문제로 변환하여 지표를 계산한 뒤, 이 값들의 평균을 내는 방식을 사용합니다. 이때 평균을 내는 전략이 중요합니다.

#### 🧠 개념 이해하기

  * **Macro Average**: 모든 클래스를 동등하게 취급합니다. 각 클래스별로 평가지표(예: F1-score)를 각각 계산한 뒤, 이들의 단순 산술 평균을 냅니다. 데이터 수가 적은 클래스의 성능에 관심이 많을 때 유용합니다.
  * **Weighted Average**: 클래스별 가중치를 부여하여 평균을 냅니다. 각 클래스의 실제 샘플 수(support)를 가중치로 사용하여 평가지표의 평균을 계산합니다. 클래스 불균형이 있을 때, 데이터 분포를 고려하여 전체 성능을 평가하고 싶을 때 적합합니다.
  * **Micro Average**: 모든 클래스의 TP, FP, FN 값을 전부 합산하여 전체 성능을 한 번에 계산합니다. 다중 클래스에서 micro-F1 스코어는 전체 데이터의 정확도(Accuracy)와 동일한 값을 가집니다.

#### 💻 코드 예시

In [8]:
from sklearn.metrics import f1_score

# 3개 클래스(0, 1, 2) 분류 예시, 클래스 2가 매우 적음 (불균형)
y_true_multi = [0, 0, 0, 0, 1, 1, 1, 2]
y_pred_multi = [0, 0, 1, 0, 1, 1, 0, 2]

# Macro F1: 각 클래스의 F1 점수 (class 0: 0.8, class 1: 0.67, class 2: 1.0)의 산술 평균
f1_macro = f1_score(y_true_multi, y_pred_multi, average='macro')

# Weighted F1: 각 클래스 F1 점수에 샘플 수(class 0: 4개, class 1: 3개, class 2: 1개) 가중치를 부여한 평균
f1_weighted = f1_score(y_true_multi, y_pred_multi, average='weighted')

print(f"Macro F1 Score: {f1_macro:.2f}")
print(f"Weighted F1 Score: {f1_weighted:.2f}")

Macro F1 Score: 0.81
Weighted F1 Score: 0.75


Macro F1은 소수 클래스(class 2)의 성능을 동등하게 반영하는 반면, Weighted F1은 다수 클래스(class 0, 1)의 성능을 더 중요하게 반영합니다.

#### ✏️ 연습문제 5

어떤 모델이 세 개의 클래스(A, B, C)를 예측한 결과가 다음과 같습니다.

  * 클래스 A: F1-score=0.9, 실제 샘플 수=80
  * 클래스 B: F1-score=0.8, 실제 샘플 수=15
  * 클래스 C: F1-score=0.5, 실제 샘플 수=5

이 모델의 **Macro F1**과 **Weighted F1** 중 어느 쪽이 더 높게 나올까요? 그 이유는 무엇일까요?

-----

### 6\. ROC Curve & AUC: 모델의 종합 성능 평가

**ROC(Receiver Operating Characteristic) Curve**와 \*\*AUC(Area Under the Curve)\*\*는 이진 분류 모델의 성능을 종합적으로 평가하는 데 가장 널리 사용되는 지표 중 하나입니다. 임계값(Threshold)의 변화에 따라 모델의 성능이 어떻게 변하는지를 시각적으로 보여줍니다.

#### 🧠 개념 이해하기

ROC Curve는 x축을 **FPR(False Positive Rate)**, y축을 **TPR(True Positive Rate)** 로 놓고 그립니다.

  * **TPR (True Positive Rate, 재현율)**: $\\frac{\\text{TP}}{\\text{TP} + \\text{FN}}$. 실제 Positive 중 모델이 Positive라고 맞춘 비율. (1에 가까울수록 좋음)
  * **FPR (False Positive Rate)**: $\\frac{\\text{FP}}{\\text{FP} + \\text{TN}}$. 실제 Negative 중 모델이 Positive라고 틀리게 예측한 비율. (0에 가까울수록 좋음)

  * **(0,1) 지점 (좌상단)**: FPR은 0이고 TPR은 1인, 완벽한 모델의 위치입니다. ROC 커브가 이 지점에 가까울수록 모델의 성능이 좋다는 의미입니다.
  * **대각선 (y=x)**: 완전한 랜덤 예측을 의미합니다. 모델의 성능이 동전 던지기와 같다는 뜻입니다. 커브가 이 대각선 아래에 있다면 랜덤 예측보다도 못한 모델입니다.

**AUC (Area Under the Curve)**
AUC는 ROC Curve 아래의 면적을 의미하며, 0과 1 사이의 값을 가집니다. 이 면적이 넓을수록(1에 가까울수록) 모델의 성능이 우수함을 나타냅니다.

  * **AUC = 1**: 완벽한 분류기.
  * **AUC = 0.5**: 랜덤 분류기 (쓸모 없음).
  * **AUC의 직관적 의미**: "랜덤하게 뽑은 Positive 샘플의 예측 점수가 랜덤하게 뽑은 Negative 샘플의 예측 점수보다 높을 확률"을 나타냅니다. AUC가 0.9라면, 90% 확률로 Positive 샘플을 더 잘 구분한다는 뜻입니다.

AUC는 임계값에 상관없이 모델의 전반적인 '분류 능력' 자체를 평가하므로, 특정 임계값에 의존하지 않는 안정적인 성능 평가가 가능합니다.

#### 💻 코드 예시

ROC Curve를 그리려면 `predict()`로 얻은 0/1 예측값이 아닌, `predict_proba()`로 얻은 'Positive일 확률'이 필요합니다.

In [9]:
import plotly.graph_objects as go
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# 예시 데이터 생성
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=1000, n_classes=2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 모델 학습 및 확률 예측
model = LogisticRegression().fit(X_train, y_train)
y_scores = model.predict_proba(X_test)[:, 1] # Positive 클래스(1)에 대한 확률
y_scores


array([6.09536672e-01, 8.68929260e-01, 5.13909465e-01, 7.79089383e-01,
       9.53723656e-01, 5.23451932e-02, 2.27636761e-01, 7.89347054e-01,
       7.37037455e-01, 3.12417654e-01, 9.66424758e-01, 6.70829492e-01,
       4.08948143e-03, 4.99306936e-01, 7.19672711e-01, 1.79605592e-01,
       6.44589971e-02, 9.51291957e-01, 8.87630911e-01, 4.78687093e-01,
       9.94347672e-01, 2.55081828e-01, 5.14401674e-02, 1.02764693e-01,
       9.78692448e-01, 4.51196086e-03, 9.99621703e-01, 9.37634035e-01,
       1.18188476e-02, 9.69791848e-01, 8.68816110e-01, 9.88422095e-01,
       6.15473106e-01, 1.47860299e-01, 3.12980543e-01, 1.65110252e-01,
       7.00945658e-02, 4.04964522e-01, 4.55256562e-02, 2.76558176e-01,
       1.86227131e-02, 9.48312870e-01, 5.57726753e-02, 4.53793669e-01,
       5.01184326e-01, 1.14111255e-02, 9.92328859e-01, 5.41306342e-03,
       8.59704303e-01, 9.90278194e-01, 7.62490429e-02, 7.65075587e-02,
       1.00222414e-02, 9.96565652e-01, 9.96836896e-01, 6.70517119e-01,
      

In [11]:
# ROC Curve 계산
fpr, tpr, thresholds = roc_curve(y_test, y_scores)
fpr, tpr, thresholds


(array([0.        , 0.        , 0.        , 0.00689655, 0.00689655,
        0.0137931 , 0.0137931 , 0.02068966, 0.02068966, 0.02758621,
        0.02758621, 0.03448276, 0.03448276, 0.04137931, 0.04137931,
        0.04827586, 0.04827586, 0.05517241, 0.05517241, 0.06206897,
        0.06206897, 0.07586207, 0.07586207, 0.08965517, 0.08965517,
        0.09655172, 0.09655172, 0.10344828, 0.10344828, 0.11034483,
        0.11034483, 0.11724138, 0.11724138, 0.12413793, 0.12413793,
        0.13103448, 0.13103448, 0.13793103, 0.13793103, 0.14482759,
        0.14482759, 0.17931034, 0.17931034, 0.19310345, 0.19310345,
        0.2137931 , 0.2137931 , 0.28275862, 0.28275862, 0.33103448,
        0.33103448, 0.34482759, 0.34482759, 0.4137931 , 0.4137931 ,
        0.43448276, 0.43448276, 0.48965517, 0.48965517, 0.55172414,
        0.55172414, 0.55862069, 0.55862069, 0.60689655, 0.60689655,
        0.64137931, 0.64137931, 0.8       , 0.8       , 0.90344828,
        0.90344828, 0.91034483, 0.91034483, 1.  

In [12]:
# AUC 계산
auc_score = roc_auc_score(y_test, y_scores)
auc_score

0.9141713014460511

In [13]:
# ROC Curve 시각화 (Plotly)
fig = go.Figure()
# ROC Curve
fig.add_trace(go.Scatter(x=fpr, y=tpr,
                         mode='lines',
                         name=f'ROC Curve (AUC = {auc_score:.4f})'))
# Random Classifier Line
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1],
                         mode='lines',
                         name='Random Classifier (AUC = 0.5)',
                         line=dict(dash='dash')))
fig.update_layout(title='ROC Curve',
                  xaxis_title='False Positive Rate (FPR)',
                  yaxis_title='True Positive Rate (TPR)')
fig.show()

print(f"모델의 AUC 점수: {auc_score:.4f}")

모델의 AUC 점수: 0.9142


#### ✏️ 연습문제 6

두 개의 고객 이탈 예측 모델 A와 B를 평가한 결과, 다음과 같은 AUC 점수를 얻었습니다.

  * 모델 A의 AUC = 0.92
  * 모델 B의 AUC = 0.78

어떤 모델의 전반적인 성능이 더 우수하다고 판단할 수 있으며, 그 이유는 무엇인가요?